Run: `pip install -r requirements.txt` to install the required libraries

`pyreadr` packge to help read .Rda files

In [ ]:
import pyreadr

`rpy2` package to execute R functions in python -> unsucessful. This needs both R and Python installed, and I do not want to install R. For the datasets in the `spls` package, you could make use of this package, or just run Rstudio, load the datasets there and export them to .csv files.

### PRIM dataset 

gathered from the website: 

https://rdrr.io/cran/PRIMsrc/man/Real.2-data.html

mRNA, miRNA and clinical variables to study the molecular characteristics of lung cancer cells. 

123 * 946 size

In [3]:
PRIM_data =  pyreadr.read_r('datasets/Real.2.rda')

In [6]:
print(PRIM_data.keys()) 

odict_keys(['Real.2'])


In [8]:
PRIM_data["Real.2"].head()

,y,delta,Age,Type,KRAS.status,EGFR.status,P53.status,hsa.miR.555,hsa.miR.223.,hsa.miR.346,...,hsa.miR.320d,ebv.miR.BART13,hsa.miR.562,hsa.miR.486.3p,hsa.miR.184,hsa.miR.214,hsa.miR.590.3p,hsa.miR.671.5p,hsa.miR.127.3p,ebv.miR.BART6.3p
rownames,,,,,,,,,,,,,,,,,,,,,
AGG600716,1.9,1,45.26,1,1,0,1,1.258403,-2.107231,2.201386,...,9.486216,8.483150,1.549601,-2.489153,3.489111,8.790909,0.964420,6.435507,8.092005,0.555592
ANO420520,4.3,0,63.81,1,1,0,1,1.427363,-2.540758,2.784025,...,10.846289,8.617958,-0.218795,-1.023582,3.426729,11.101470,0.832330,7.323581,8.675192,-1.187546
ARC270517,5.1,0,78.44,3,0,0,1,0.422615,2.444766,2.633147,...,9.685072,9.975355,-0.196234,-0.755116,4.127585,9.290808,-1.149080,7.588815,7.864244,-1.106565
AVI260916,7.7,0,76.39,1,0,0,0,0.703600,-2.315074,2.158156,...,10.816104,9.134508,0.940189,-0.668845,4.241370,10.363584,0.370006,7.414035,8.586668,-1.608089
AZE450213,3.5,1,59.84,1,0,0,0,1.535154,-2.888803,2.233118,...,9.733772,9.299710,1.206737,-0.908456,3.850926,10.912283,0.972962,7.215852,7.008386,0.348046


In [22]:
import rpy2
print(rpy2.__version__)


3.5.16


### Prostate data 

#### _This does not work - please get the `spls` package in R and then export the datasets to csv._

from the `spls` package in R.

Although it is unable to import (probably because it is saved as a R list or something like that), it has a pretty good dimension: 102 * 6033. Also has an outcome class (normal vs. tumor). 

sPLS paper (Chung and Keles) that already did some analysis on this: https://pages.stat.wisc.edu/~keles/Papers/C_SPLS.pdf

There are other datasets in the `spls` package. Such as: `lymphona` (62 * 4026), `mice` (60 * 145), `yeast` (542 * 106)

The below is the failed attempt at getting the Rdata here

In [9]:
prostate_data =  pyreadr.read_r('datasets/prostate.Rdata')

In [12]:
print(prostate_data.keys())

odict_keys([])


In [15]:
prostate_data[""]

KeyError: ''

### DOROTHEA data. 

This is to study sensitivity to a drug (which receptor reacts to the drug).

It's a binary dataset. 800 * 100k. 

50k features are actual features, while the other 50k are noise features, because this dataset was designed to study variable selection.

DOROTHEA data is part of the NIPS 2003 feature selection challenge. It also contains ARCENE data. I am not completely sure, but it seems like these datasets are combinations of publicly available dataset, so this may mean that this is rather artificial. 

Also, for the challenge, many random noise features were added. But I cannot find information online on which features are actually important - I think this can be interesting to study, though.

I got the below function from here :https://github.com/Tuyki/TT_RNN/blob/master/Experiment_NIPS2003.py

In [35]:

def get_dorothea_data(filename, mode='online'):
    """
    A function that reads in the original dorothea data in sparse form of feature:value
    and transform them into matrix form.
    # Arguments:
    filename: the url to either the dorothea_train.data or dorothea_valid.data
    mode: either 'text' for unpacked file; 'gz' for .gz file; or 'online' to download from the UCI repo
    # Return:
    the dexter data in matrix form.
    """
    if mode == 'txt':
        with open(filename) as f:
            readin_list = f.readlines()
    elif mode == 'gz':
        with gzip.open(filename) as f:
            readin_list = f.readlines()

    def to_dense_dorothea(string_array):
        n = len(string_array)
        inds = np.zeros(n, dtype='int32')
        ret = np.zeros(100001, dtype='int32')
        for i in range(n):
            this_split = string_array[i].split(' ')
            inds[i] = int(this_split[0])
        ret[inds] = 1
        return ret

    N = len(readin_list)
    dat = [None]*N

    for i in range(N):
        dat[i] = to_dense_dorothea(readin_list[i].split(' ')[1:-1])[None, :]

    dat = np.concatenate(dat, axis=0).astype('float32')
    return dat[:, 1::]  # the first column all zero


In [36]:
import numpy as np

In [37]:
X_train = get_dorothea_data("datasets/DOROTHEA/dorothea_train.data", mode='txt')


In [38]:
X_train.shape

(800, 100000)

In [41]:
# show first five rows of X_train
X_train[:5, :]

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [42]:
# show the unique values of X_train
np.unique(X_train)

array([0., 1.], dtype=float32)

In [43]:
Y_train = np.loadtxt("datasets/DOROTHEA/dorothea_train.labels")

In [45]:
np.unique(Y_train)

array([-1.,  1.])

### ARCENE dataset

ARCENE was obtained by merging three mass-spectrometry datasets to obtain enough training and test data for a benchmark. The original features indicate the abundance of proteins in human sera having a given mass value. Based on those features one must separate cancer patients from healthy patients. We added a number of distractor feature called 'probes' having no predictive power.

In [46]:
arcene_X = np.loadtxt("datasets/ARCENE/arcene_train.data")

In [49]:
arcene_X[:5,:]

array([[  0.,  71.,   0., ...,   0.,   0., 524.],
       [  0.,  41.,  82., ...,   0., 284., 423.],
       [  0.,   0.,   1., ...,   0.,  34., 508.],
       [  0.,  56.,  44., ...,   0.,   0., 469.],
       [105.,   0., 141., ...,   0.,   0., 354.]])

In [50]:
arcene_X.shape

(100, 10000)

### Toxicity data 

(https://archive.ics.uci.edu/dataset/728/toxicity-2)

Each row is a molecule, and each column is a measurement (molecular descriptor). Only 13 features are important - i guess in determining whether the molecule is toxic or not. And this list of important features is available. 

In [51]:
import pandas as pd

# Read the CSV file into a DataFrame
data = pd.read_csv('datasets/toxicity-2/data.csv')

# Print the DataFrame
print(data)

     MATS3v  nHBint10  MATS3s  MATS3p  nHBDon_Lipinski  minHBint8  MATS3e  \
0    0.0908         0  0.0075  0.0173                0     0.0000 -0.0436   
1    0.0213         0  0.1144 -0.0410                0     0.0000  0.1231   
2    0.0018         0 -0.0156 -0.0765                2     0.0000 -0.1138   
3   -0.0251         0 -0.0064 -0.0894                3     0.0000 -0.0747   
4    0.0135         0  0.0424 -0.0353                0     0.0000 -0.0638   
..      ...       ...     ...     ...              ...        ...     ...   
166 -0.0960         0 -0.0478 -0.0840                2     0.0000 -0.0739   
167 -0.0064         1 -0.1222  0.0013                1     0.0000 -0.1873   
168  0.0096         2 -0.1846  0.0058                1     0.0000 -0.1293   
169 -0.0736         2 -0.1267 -0.0345                2     0.5346 -0.0361   
170 -0.1864         0 -0.0020 -0.1968                1     8.1414 -0.0529   

     MATS3c  minHBint2  MATS3m  ...   WTPT-4   WTPT-5  ETA_EtaP_L  ETA_EtaP

In [52]:
data.shape

(171, 1204)

### PANCAN dataset 

(obtained via https://archive.ics.uci.edu/dataset/401/gene+expression+cancer+rna+seq)

This collection of data is part of the RNA-Seq (HiSeq) PANCAN data set, it is a random extraction of gene expressions of patients having different types of tumor: BRCA, KIRC, COAD, LUAD and PRAD.




In [56]:
# Read the CSV file into a DataFrame
pancan_data = pd.read_csv('datasets/TCGA-PANCAN-HiSeq-801x20531/data.csv')


In [54]:
pancan_data.shape

(801, 20532)

In [55]:
pancan_data.head()

,Unnamed: 0,gene_0,gene_1,gene_2,gene_3,gene_4,gene_5,gene_6,gene_7,gene_8,...,gene_20521,gene_20522,gene_20523,gene_20524,gene_20525,gene_20526,gene_20527,gene_20528,gene_20529,gene_20530
0,sample_0,0.0,2.017209,3.265527,5.478487,10.431999,0.0,7.175175,0.591871,0.0,...,4.926711,8.210257,9.723516,7.220030,9.119813,12.003135,9.650743,8.921326,5.286759,0.0
1,sample_1,0.0,0.592732,1.588421,7.586157,9.623011,0.0,6.816049,0.000000,0.0,...,4.593372,7.323865,9.740931,6.256586,8.381612,12.674552,10.517059,9.397854,2.094168,0.0
2,sample_2,0.0,3.511759,4.327199,6.881787,9.870730,0.0,6.972130,0.452595,0.0,...,5.125213,8.127123,10.908640,5.401607,9.911597,9.045255,9.788359,10.090470,1.683023,0.0
3,sample_3,0.0,3.663618,4.507649,6.659068,10.196184,0.0,7.843375,0.434882,0.0,...,6.076566,8.792959,10.141520,8.942805,9.601208,11.392682,9.694814,9.684365,3.292001,0.0
4,sample_4,0.0,2.655741,2.821547,6.539454,9.738265,0.0,6.566967,0.360982,0.0,...,5.996032,8.891425,10.373790,7.181162,9.846910,11.922439,9.217749,9.461191,5.110372,0.0


In [58]:
# get the name of columns of pancan_data
pancan_data.columns

Index(['Unnamed: 0', 'gene_0', 'gene_1', 'gene_2', 'gene_3', 'gene_4',
       'gene_5', 'gene_6', 'gene_7', 'gene_8',
       ...
       'gene_20521', 'gene_20522', 'gene_20523', 'gene_20524', 'gene_20525',
       'gene_20526', 'gene_20527', 'gene_20528', 'gene_20529', 'gene_20530'],
      dtype='object', length=20532)

### SECOM data 

for semiconductor manufacturing (https://archive.ics.uci.edu/dataset/179/secom)

This dataset is about semiconductor manucatruing process. Engineers monitor different signals to make sure that the manufacturing takes place without failures. But some of these signals are irrelevant, while others are more important. The features are the signals here.



In [62]:
secom_data = np.loadtxt("datasets/secom/secom.data")

In [63]:
secom_data.shape

(1567, 590)

In [64]:
secom_data[:5,:]

array([[3.0309300e+03, 2.5640000e+03, 2.1877333e+03, ...,           nan,
                  nan,           nan],
       [3.0957800e+03, 2.4651400e+03, 2.2304222e+03, ..., 2.0100000e-02,
        6.0000000e-03, 2.0820450e+02],
       [2.9326100e+03, 2.5599400e+03, 2.1864111e+03, ..., 4.8400000e-02,
        1.4800000e-02, 8.2860200e+01],
       [2.9887200e+03, 2.4799000e+03, 2.1990333e+03, ..., 1.4900000e-02,
        4.4000000e-03, 7.3843200e+01],
       [3.0322400e+03, 2.5028700e+03, 2.2333667e+03, ..., 1.4900000e-02,
        4.4000000e-03, 7.3843200e+01]])

In [66]:
len(np.unique(secom_data))

213026

In [67]:
# count the number of missing values in each column
missing_values = np.isnan(secom_data).sum(axis=0)

In [68]:
missing_values

array([   6,    7,   14,   14,   14,   14,   14,    9,    2,    2,    2,
          2,    2,    3,    3,    3,    3,    3,    3,   10,    0,    2,
          2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    1,
          1,    1,    1,    1,    1,    1,    1,   24,   24,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    4,    4,
          4,    4,    4,    4,    7,    6,    6,    6,    7,    7,    7,
          6,    6,    6,    6,    6,    6,  794,  794,    6,   24,   24,
         24,   24,   24,   24,   24,   24,    1,   12, 1341,    0,    0,
          0,   51,   51,    6,    2,    2,    6,    6,    6,    6,    6,
          6,    6,    6,    6,    2,    2,    6,    6,    6,    6, 1018,
       1018, 1018,  715,    0,    0,    0,    0,    0,   24,    0,    0,
          9,    9,    9,    9,    9,    9,    9,    9,    9,    9,    9,
          8,    8,    8,    5,    6,    7,   14,   14,   14,   14,   14,
          9,    2,    2,    2,    2,    2,    3,   